In [1]:
# import libraries (you may add additional imports but you may not have to)
import numpy as np
import pandas as pd
from scipy.sparse import csr_matrix
from sklearn.neighbors import NearestNeighbors
import matplotlib.pyplot as plt

In [2]:
# get data files
!wget https://cdn.freecodecamp.org/project-data/books/book-crossings.zip

!unzip book-crossings.zip

books_filename = 'BX-Books.csv'
ratings_filename = 'BX-Book-Ratings.csv'

--2024-04-09 13:48:54--  https://cdn.freecodecamp.org/project-data/books/book-crossings.zip
Resolving cdn.freecodecamp.org (cdn.freecodecamp.org)... 172.67.70.149, 104.26.3.33, 104.26.2.33, ...
Connecting to cdn.freecodecamp.org (cdn.freecodecamp.org)|172.67.70.149|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 26085508 (25M) [application/zip]
Saving to: ‘book-crossings.zip.2’

book-crossings.zip. 100%[===================>]  24.88M   109MB/s    in 0.2s    

2024-04-09 13:48:54 (109 MB/s) - ‘book-crossings.zip.2’ saved [26085508/26085508]

Archive:  book-crossings.zip
replace BX-Book-Ratings.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
  inflating: BX-Book-Ratings.csv     
replace BX-Books.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
  inflating: BX-Books.csv            
replace BX-Users.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
  inflating: BX-Users.csv            


After importing and cleaning the data, use NearestNeighbors from sklearn.neighbors to develop a model that shows books that are similar to a given book. The Nearest Neighbors algorithm measures the distance to determine the “closeness” of instances.

In [3]:
# import csv data into dataframes
df_books = pd.read_csv(
    books_filename,
    encoding = "ISO-8859-1",
    sep=";",
    header=0,
    names=['isbn', 'title', 'author'],
    usecols=['isbn', 'title', 'author'],
    dtype={'isbn': 'str', 'title': 'str', 'author': 'str'})

df_ratings = pd.read_csv(
    ratings_filename,
    encoding = "ISO-8859-1",
    sep=";",
    header=0,
    names=['user', 'isbn', 'rating'],
    usecols=['user', 'isbn', 'rating'],
    dtype={'user': 'int32', 'isbn': 'str', 'rating': 'float32'})

In [4]:
df_books

,isbn,title,author
0,0195153448,Classical Mythology,Mark P. O. Morford
1,0002005018,Clara Callan,Richard Bruce Wright
2,0060973129,Decision in Normandy,Carlo D'Este
3,0374157065,Flu: The Story of the Great Influenza Pandemic...,Gina Bari Kolata
4,0393045218,The Mummies of Urumchi,E. J. W. Barber
...,...,...,...
271374,0440400988,There's a Bat in Bunk Five,Paula Danziger
271375,0525447644,From One to One Hundred,Teri Sloat
271376,006008667X,Lily Dale : The True Story of the Town that Ta...,Christine Wicker
271377,0192126040,Republic (World's Classics),Plato


If you graph the dataset (optional), you will notice that most books are not rated frequently. To ensure statistical significance, remove from the dataset users with less than 200 ratings and books with less than 100 ratings.

In [13]:

# Assuming df is your pandas DataFrame with book rating data
# It should have columns: 'isbn', 'title', 'author', 'user', 'rating'

# Group by ISBN and count the number of ratings for each book
df=pd.merge(df_books, df_ratings, on='isbn')
df.value_counts('isbn').sort_values()[:20]


isbn
0671027344    16
0684833395    22
0552998486    23
076790592X    23
0451191145    23
0451204530    23
039592720X    23
0140042598    24
1573227331    25
0316781142    25
0553375407    25
0446525502    25
006019491X    26
0385475721    27
0312982518    27
0099771519    27
0312265859    27
0446530522    28
0061015725    28
0609610597    28
Name: count, dtype: int64

In [14]:
# count how many records per user
user_counts = df_ratings['user'].value_counts()
# count how many records per book
book_counts = df_ratings['isbn'].value_counts()

#filter data
df_ratings=df_ratings[~df_ratings['user'].isin(user_counts[user_counts < 200].index)]
df_ratings=df_ratings[~df_ratings['isbn'].isin(book_counts[book_counts < 100].index)]

df_ratings

,user,isbn,rating
45618,11676,0060502258,8.0
45706,11676,0060928336,0.0
45713,11676,0060930535,0.0
45725,11676,0060934417,0.0
45776,11676,0060976845,0.0
...,...,...,...
990475,238120,080410526X,10.0
990477,238120,0804106304,0.0
990478,238120,080410753X,0.0
990567,238120,0971880107,0.0


In [15]:
# merge two dataframe by isbn
df_all=pd.merge(df_books, df_ratings, on='isbn')
#remove if a row has the same combination of values in the "title" and "user" columns as another row
df_all = df_all.drop_duplicates(["title", "user"])
df_all

,isbn,title,author,user,rating
0,0440234743,The Testament,John Grisham,11676,9.0
1,0440234743,The Testament,John Grisham,16795,0.0
2,0440234743,The Testament,John Grisham,23768,0.0
3,0440234743,The Testament,John Grisham,35859,0.0
4,0440234743,The Testament,John Grisham,52584,0.0
...,...,...,...,...,...
1224,0446607657,When the Wind Blows,James Patterson,185233,5.0
1225,0446607657,When the Wind Blows,James Patterson,196077,0.0
1226,0446607657,When the Wind Blows,James Patterson,198711,0.0
1227,0446607657,When the Wind Blows,James Patterson,204864,0.0


In [16]:
book_ratings_pivot=df_all.pivot(index='title',columns='user',values='rating').fillna(0)
book_ratings_pivot

user,11676,16795,21014,23768,35859,43246,52584,55492,60244,76352,...,102967,135149,153662,185233,196077,198711,204864,230522,232131,238120
title,,,,,,,,,,,,,,,,,,,,,
1st to Die: A Novel,10.0,9.0,0.0,0.0,7.0,9.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,8.0,0.0
A Is for Alibi (Kinsey Millhone Mysteries (Paperback)),0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
A Map of the World,8.0,9.0,0.0,0.0,0.0,0.0,0.0,0.0,8.0,0.0,...,0.0,0.0,0.0,0.0,7.0,0.0,0.0,0.0,0.0,0.0
A Painted House,8.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,8.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
A Prayer for Owen Meany,10.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,9.0,0.0,9.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Where the Heart Is (Oprah's Book Club (Paperback)),10.0,8.0,0.0,0.0,0.0,0.0,8.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,9.0,0.0
While I Was Gone,0.0,0.0,0.0,0.0,0.0,9.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
White Oleander : A Novel (Oprah's Book Club),0.0,0.0,7.0,0.0,0.0,0.0,0.0,0.0,8.0,0.0,...,0.0,0.0,9.0,0.0,9.0,0.0,0.0,0.0,0.0,0.0


In [17]:
# turn in to numpay array
matrix = book_ratings_pivot.values
matrix.shape


(99, 21)

In [18]:
from sklearn.preprocessing import StandardScaler

model_knn=NearestNeighbors(metric='cosine',algorithm='brute')
model_knn.fit(matrix)

NearestNeighbors(algorithm='brute', metric='cosine')

Create a function named get_recommends that takes a book title (from the dataset) as an argument and returns a list of 5 similar books with their distances from the book argument.

This code:
```
get_recommends("The Queen of the Damned (Vampire Chronicles (Paperback))")
```
should return:
```
[
  'The Queen of the Damned (Vampire Chronicles (Paperback))',
  [
    ['Catch 22', 0.793983519077301],
    ['The Witching Hour (Lives of the Mayfair Witches)', 0.7448656558990479],
    ['Interview with the Vampire', 0.7345068454742432],
    ['The Tale of the Body Thief (Vampire Chronicles (Paperback))', 0.5376338362693787],
    ['The Vampire Lestat (Vampire Chronicles, Book II)', 0.5178412199020386]
  ]
]

```



In [19]:
# function to return recommended books - this will be tested
def get_recommends(book = ""):

    # extract the vector from pivot table
    x = book_ratings_pivot.loc[book].array.reshape(1,-1)
    # Use the Nearest Neighbors model to find nearest neighbors
    distances,indices = model_knn.kneighbors(x,n_neighbors=6)


    # Create a list of recommended books with their distances
    recommended_books = []
    for distance,indice in zip(distances[0],indices[0]):
      if distance!=0:
        r_book=book_ratings_pivot.index[indice]
        recommended_books.append([r_book,distance])
    return [book, recommended_books[::-1]] # reverses the list [::-1]



In [20]:
books = get_recommends("Where the Heart Is (Oprah's Book Club (Paperback))")
print(books)

def test_book_recommendation():
  test_pass = True
  recommends = get_recommends("Where the Heart Is (Oprah's Book Club (Paperback))")
  if recommends[0] != "Where the Heart Is (Oprah's Book Club (Paperback))":
    test_pass = False
  recommended_books = ["I'll Be Seeing You", 'The Weight of Water', 'The Surgeon', 'I Know This Much Is True']
  recommended_books_dist = [0.8, 0.77, 0.77, 0.77]
  for i in range(2):
    if recommends[1][i][0] not in recommended_books:
      test_pass = False
    if abs(recommends[1][i][1] - recommended_books_dist[i]) >= 0.05:
      test_pass = False
  if test_pass:
    print("You passed the challenge! 🎉🎉🎉🎉🎉")
  else:
    print("You haven't passed yet. Keep trying!")

test_book_recommendation()

["Where the Heart Is (Oprah's Book Club (Paperback))", [['Violets Are Blue', 0.242661], ['The Testament', 0.23662043], ['The Street Lawyer', 0.23472488], ['The Da Vinci Code', 0.22260845], ['Hannibal', 0.17240119]]]
You haven't passed yet. Keep trying!
